<a href="https://colab.research.google.com/github/fdeiab/loan-default-prediction/blob/main/Loan_Default_Predicition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

Loan approval is a critical decision for any lending institution. One of two scenarios may happen, the bank loses out on potential income by rejecting a loan to an individual or a company. Or, they lend a loan to a party exhibiting risky behaviour where heavy losses. This uncertainty is why it is a good idea that banks utilize good loan default prediction models as they stand much to gain. 

**The major goal of this notebook is to predict whether a loan will default.**